In [1]:
# Basic libraries
import time
import argparse

from os.path import abspath
from os.path import dirname as up
import numpy as np
import sys
import h5py

# Insert path to pybf library to system path
print(up(up(up(abspath("__file__")))))
path_to_lib = up(up(up(abspath("__file__"))))
print(path_to_lib)
sys.path.insert(0, path_to_lib)

#Import libraries and functions

from pybf.pybf.io_interfaces import DataLoader
from pybf.pybf.signal_processing import demodulate_decimate
from pybf.pybf.signal_processing import interpolate_modulate
from pybf.pybf.signal_processing import filter_band_pass
from pybf.scripts.beamformer_cartesian_realtime import BFCartesianRealTime
from pybf.pybf.transducer import Transducer
from pybf.pybf.image_settings import ImageSettings
from pybf.pybf.visualization import plot_image
from pybf.scripts.beamformer_cartesian import beamformer_cartesian

/home/sem21f11/Desktop
/home/sem21f11/Desktop


/home/sem21f11/miniconda3/envs/pybf_env/lib/python3.6/importlib/__init__.py:126: MatplotlibDeprecationWarning: 
The matplotlib.backends.backend_qt4agg backend was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
  return _bootstrap._gcd_import(name[level:], package, level)


In [2]:
def get_original_rf_data(path_to_dataset):
    
    # Read file
    data_loader_obj = DataLoader(path_to_dataset)
    
    rf_data_shape = (data_loader_obj.num_of_acq_per_frame,) + data_loader_obj.get_rf_data(0, 0).shape
    print('RF data shape: ', rf_data_shape)

    rf_data = np.zeros(rf_data_shape)
    for i in range(rf_data.shape[0]):
        rf_data[i, :, :] = data_loader_obj.get_rf_data(0, i)
        
    return rf_data

In [3]:
rf_original_data = get_original_rf_data("../datasets/archive/archive_to_download/database/experiments/contrast_speckle/rf_dataset.hdf5")

print(np.amax(rf_original_data))
print(np.amin(rf_original_data))

RF data shape:  (75, 128, 3328)
1.0
-0.989680826663971


In [4]:
# Convert floating point numbers to integers in a range set by the bit-width of an ADC

def convert_to_int(rf_original_data, bits_ADC, full_range):


    # Adjust range of original data to positive values
    rf_range_data = rf_original_data + 1

    bits_ADC = bits_ADC
    full_range = full_range

    # Multiply all values with the following
    mult_with = ((2**bits_ADC)-1)/full_range

    mult_data = rf_range_data*mult_with
    #print(mult_data.shape)
    flatten_mult_data = mult_data.flatten()
    #print(flatten_mult_data.shape)

    # Convert to integers
    int_converted_data = np.zeros(rf_original_data.shape, dtype='int32')
    flatten_conv_data = int_converted_data.flatten()
    #print(flatten_conv_data.shape)

    i = 0
    for sample in flatten_mult_data:
        flatten_conv_data[i] = int(np.round(sample))
        i = i+1
        #sys.stdout.write('\r')
        #sys.stdout.write("{} out of {}".format(i, flatten_mult_data.shape[0]))

    int_converted_data = flatten_conv_data.reshape(rf_original_data.shape)
    print("Converted data shape: {}".format(int_converted_data.shape))

    #print("Done converting to integers.")
    return int_converted_data

In [5]:
# Used for Decimation-Demodulation -> Re-quantization

def convert_to_int_2(rf_original_data, bits_ADC, full_range):


    # Adjust range of original data to positive values
    #rf_range_data = rf_original_data + 1

    bits_ADC = bits_ADC
    full_range = full_range

    # Multiply all values with the following
    mult_with = ((2**bits_ADC)-1)/full_range

    mult_data = rf_original_data*mult_with
    #print(mult_data.shape)
    flatten_mult_data = mult_data.flatten()
    #print(flatten_mult_data.shape)

    # Convert to integers
    int_converted_data = np.zeros(rf_original_data.shape, dtype='int32')
    flatten_conv_data = int_converted_data.flatten()
    #print(flatten_conv_data.shape)

    i = 0
    for sample in flatten_mult_data:
        flatten_conv_data[i] = int(np.round(sample))
        i = i+1
        #sys.stdout.write('\r')
        #sys.stdout.write("{} out of {}".format(i, flatten_mult_data.shape[0]))

    int_converted_data = flatten_conv_data.reshape(rf_original_data.shape)
    print("Converted data shape: {}".format(int_converted_data.shape))

    #print("Done converting to integers.")
    return int_converted_data

In [6]:
# Convert integers to floating point numbers
# "bits" represents the bit-width representation of the numbers, extracted by the range
# e.g. if range 0-511, then bits = 9

def convert_to_float(input_array, bits):
    
    divider = ((2**bits)-1)/2
    rf_range_data = input_array/divider
    rf_data = rf_range_data - 1
    
    return rf_data
    

In [7]:
# LAVA function

def LAVA(threshold, input_array):
    
    # Steps
    thres_diff = threshold  # The threshold value for difference in order to keep the next value

    register = 0

    kept_values = []
    indexes = []

    for i in range(input_array.shape[0]):
        acq_kept_values = []
        acq_indexes = []
        for k in range(input_array.shape[1]):
            register = input_array[i][k][0]
            ch_kept_values = []
            ch_indexes = []
            for index in range(1, input_array.shape[2]):
                if (abs(input_array[i][k][index] - register) > thres_diff):
                    register = input_array[i][k][index]
                    ch_kept_values.append(input_array[i][k][index])
                    ch_indexes.append(index)
            acq_kept_values.append(ch_kept_values)
            acq_indexes.append(ch_indexes)
            #sys.stdout.write('\r')
            #sys.stdout.write("Done with channel {} of acq {}".format(k, i))
            
        kept_values.append(acq_kept_values)
        indexes.append(acq_indexes)

    print("Done with LAVA compression.")

    
    return kept_values, indexes

In [8]:
# Function to perform requantization and inverse requantization
# Name input_array either resolution_distorsion or contrast_speckle
def requantization(bits, input_array, old_min_1, old_max_1, new_min_1, new_max_1, old_min_2, old_max_2, new_min_2, new_max_2, old_min_3, old_max_3, new_min_3, new_max_3):

    # Requantize data in x bits
    #print(input_array.shape)

    # Ranges
    newmax_1 = new_max_1
    newmin_1 = new_min_1
    newran_1 = newmax_1 - newmin_1

    oldmax_1 = old_max_1
    oldmin_1 = old_min_1
    oldran_1 = oldmax_1 - oldmin_1

    newmax_2 = new_max_2
    newmin_2 = new_min_2
    newran_2 = newmax_2 - newmin_2

    oldmax_2 = old_max_2
    oldmin_2 = old_min_2
    oldran_2 = oldmax_2 - oldmin_2

    newmax_3 = new_max_3
    newmin_3 = new_min_3
    newran_3 = newmax_3 - newmin_3

    oldmax_3 = old_max_3
    oldmin_3 = old_min_3
    oldran_3 = oldmax_3 - oldmin_3


    # Requantize data
    requant = np.zeros((input_array.shape), dtype=np.int32)
    flat_requant = requant.flatten()
    #print(flat_requant.shape)
    index = 0
    for sample in input_array.flatten():
        if ((sample >= oldmin_1) and (sample < oldmax_1)):
            flat_requant[index] = int(np.round((((sample-oldmin_1)*newran_1)/oldran_1) + newmin_1))

        elif ((sample >= oldmin_2) and (sample < oldmax_2)):
            flat_requant[index] = int(np.round((((sample-oldmin_2)*newran_2)/oldran_2) + newmin_2))

        elif ((sample >= oldmin_3) and (sample <= oldmax_3)):
            flat_requant[index] = int(np.round((((sample-oldmin_3)*newran_3)/oldran_3) + newmin_3))    

        index = index + 1
        #sys.stdout.write('\r')
        #sys.stdout.write("{} out of {}".format(index, input_array.shape[0]))

    #print(flat_requant.shape)

    requantized_data = flat_requant.reshape(input_array.shape)
    
    return requantized_data

    

In [9]:
def inverse_requantization(requantized_data, old_min_1, old_max_1, new_min_1, new_max_1, old_min_2, old_max_2, new_min_2, new_max_2, old_min_3, old_max_3, new_min_3, new_max_3):

# Convert back to 12-bits by performing the inverse requantization

    # Ranges
    newmax_1 = old_max_1
    newmin_1 = old_min_1
    newran_1 = newmax_1 - newmin_1

    oldmax_1 = new_max_1
    oldmin_1 = new_min_1
    oldran_1 = oldmax_1 - oldmin_1

    newmax_2 = old_max_2
    newmin_2 = old_min_2
    newran_2 = newmax_2 - newmin_2

    oldmax_2 = new_max_2
    oldmin_2 = new_min_2
    oldran_2 = oldmax_2 - oldmin_2

    newmax_3 = old_max_3
    newmin_3 = old_min_3
    newran_3 = newmax_3 - newmin_3

    oldmax_3 = new_max_3
    oldmin_3 = new_min_3
    oldran_3 = oldmax_3 - oldmin_3

    # Inversely Requantize data
    inv_requant = np.zeros((requantized_data.shape), dtype=np.int32)
    flat_inv_requant = inv_requant.flatten()
    #print(flat_inv_requant.shape)
    index = 0
    for sample in requantized_data.flatten():
        if ((sample >= oldmin_1) and (sample < oldmax_1)):
            flat_inv_requant[index] = int(np.round((((sample-oldmin_1)*newran_1)/oldran_1) + newmin_1))

        elif ((sample >= oldmin_2) and (sample < oldmax_2)):
            flat_inv_requant[index] = int(np.round((((sample-oldmin_2)*newran_2)/oldran_2) + newmin_2))

        elif ((sample >= oldmin_3) and (sample <= oldmax_3)):
            flat_inv_requant[index] = int(np.round((((sample-oldmin_3)*newran_3)/oldran_3) + newmin_3))    

        index = index + 1
        #sys.stdout.write('\r')
        #sys.stdout.write("{} out of {}".format(index, requantized_data.shape[0]))


    #print(flat_inv_requant.shape)

    inv_requantized_data = flat_inv_requant.reshape(requantized_data.shape)
    return inv_requantized_data




## Combinations of 2 Algorithms 

In [10]:
# 1
# ADC -> LAVA -> Decimation/Demodulation

# Interpolation/Modulation -> Interpolation

def LAVA_Dec_Dem ():
    
    dataset = 'contrast_speckle'
    threshold = 0.00193345

    f_sampling = 20.832 * (10 ** 6)
    f_carrier = 5.2083 * (10 ** 6)

    # Get input data
    rf_original_data = get_original_rf_data("../datasets/archive/archive_to_download/database/experiments/" + dataset + "/rf_dataset.hdf5")

    # LAVA
    kept_val_LAVA, indexes_LAVA = LAVA(threshold, rf_original_data)

    # Recontructed LAVA array
    range_of_samples = [range(0,rf_original_data.shape[2])]
    reconstr_data = np.zeros((75, 128, 3328), dtype=np.float32)
    no_samples = 0
    no_indexes = 0


    # Convert individual channel acquisitions from list to array
    for i in range(len(kept_val_LAVA)):
        for k in range(len(kept_val_LAVA[0])):

            # Find perfect divider for each length
            for n in range(16,0,-1):

                if (len(kept_val_LAVA[i][k])%n == 0):
                    compression_ratio = n
                    #print("Compression ratio: {}".format(compression_ratio))
                    break

            compr_data_values = []

            current_kept_chann = np.zeros((1,len(kept_val_LAVA[i][k])))
            current_kept_chann[0] = np.array(kept_val_LAVA[i][k], dtype=object)

            # Demodulate and decimate
            compr_data_values.append(demodulate_decimate(current_kept_chann, f_sampling, f_carrier, compression_ratio))
            arr_data_values = compr_data_values[0][0]

            demod_values = np.zeros((1, arr_data_values.shape[0]),dtype=np.complex64)
            demod_values[0] = arr_data_values
            
            # Calculate number of samples transmitted
            no_samples = no_samples + arr_data_values.shape[0]
            no_indexes = no_indexes + len(indexes_LAVA[i][k])

            # Interpolate and modulate

            inter_data = interpolate_modulate(demod_values, f_sampling/compression_ratio, f_carrier, compression_ratio)

            inter_data_real = inter_data.real

            # Inverse LAVA / Interpolation
            reconstr_data[i][k] = np.interp(range_of_samples, indexes_LAVA[i][k], inter_data_real[0])       
        
    # Calculate compression ratio
    # Transfer demodulated data and all the indexes as they are after LAVA
    initial_number_of_bits = (rf_original_data.size)*12
    compressed_number_of_bits = 12*2*(no_samples) + 12*(no_indexes)
    
    # Compression ratio
    compr_ratio = (compressed_number_of_bits/initial_number_of_bits)*100
    print("LAVA_Dec_Dem is {:.2f}% of the original data.".format(compr_ratio))
    
    
    # Beamform and plot the compressed image

    dataset_path = "../datasets/archive/archive_to_download/database/experiments/contrast_speckle/rf_dataset.hdf5"

    data_loader_obj = DataLoader(dataset_path)

    ### Specify Image settings and create corresponding object ###

    img_res = [400, 600]
    image_x_range = [-0.019, 0.019]
    image_z_range = [0.005, 0.05]

    db_range = 50

    LATERAL_PIXEL_DENSITY_DEFAULT = 5

    img_config = ImageSettings(image_x_range[0],
                               image_x_range[1],
                               image_z_range[0],
                               image_z_range[1],
                               LATERAL_PIXEL_DENSITY_DEFAULT,
                               data_loader_obj.transducer)

    ### Specify preprocessing parameters for RF data ###

    decimation_factor = 1
    interpolation_factor = 10

    ### Specify TX strategy and Apodization parameters ###

    start_time = 0
    correction_time_shift = 0

    alpha_fov_apod = 40

    ### Specify Sampling Frequency ###

    SAMPLING_FREQ = 20.832 * (10 ** 6)

    filters_params = [1 * 10 **6, 8 * 10 **6, 0.5 * 10 **6]

    bf = BFCartesianRealTime(data_loader_obj.f_sampling,
                             data_loader_obj.tx_strategy,
                             data_loader_obj.transducer,
                             decimation_factor,
                             interpolation_factor,
                             img_res,
                             img_config,
                             start_time=start_time,
                             correction_time_shift=correction_time_shift,
                             alpha_fov_apod=alpha_fov_apod,
                             bp_filter_params=filters_params,
                             envelope_detector='hilbert',
                             picmus_dataset=True)


    # Beamforming
    img_data = bf.beamform(reconstr_data, numba_active=True)

    _ = plot_image(np.abs(img_data), 
                   scatters_coords_xz=None,
                   elements_coords_xz=None,
                   framework='plotly',
                   title='LAVA_Dec_Dem: {:.2f}% of the original data'.format(compr_ratio),
                   image_x_range=image_x_range,
                   image_z_range=image_z_range,
                   db_range=db_range,
                   colorscale='Greys',
                   save_fig=True, 
                   show=True,
                   path_to_save='./LAVA_Dec_Dem')
    
    # Prepare dataset to extract metrics of resolution and contrast

    # Read info from file
    f_d = h5py.File("../datasets/archive/archive_to_download/database/experiments/"+dataset+"/"+dataset+"_expe_dataset_rf.hdf5", "r")

    attributes = f_d.attrs

    US = f_d['US']
    attributes = US.attrs

    US_DATASET0000 = US['US_DATASET0000']
    attributes = US_DATASET0000.attrs

    type_s = attributes.__getitem__('type')

    attributes = US_DATASET0000.attrs

    keys_1 = list(f_d.keys());

    US = f_d['US']

    US_DATASET0000 = US['US_DATASET0000']

    data = US_DATASET0000['data']

    imag_n = data['imag']

    real_n = data['real'][()]

    PRF = US_DATASET0000['PRF']

    angles = US_DATASET0000['angles']

    initial_time = US_DATASET0000['initial_time']

    modulation_frequency = US_DATASET0000['modulation_frequency']

    probe_geometry = US_DATASET0000['probe_geometry']

    sampling_frequency = US_DATASET0000['sampling_frequency']

    sound_speed = US_DATASET0000['sound_speed']

    # Create dataset suitable for calculating metrics

    imag_np = np.zeros((75, 128, 3328))

    # Creation of dataset

    out_filename = out_filename = './LAVA_Dec_Dem/LAVA_Dec_Dem_th{}.hdf5'.format(threshold)
    comp_file = h5py.File(out_filename,'w')
    comp_file.close()
    comp_file = h5py.File(out_filename,'a')

    # Complete the structure

    comp_file.attrs.create('version', [b'v.0.0.40'], dtype='|S9')

    US_path = 'US'
    US_group = comp_file.require_group(US_path)

    US_dataset_group = US_group.require_group('US_DATASET0000')

    US_dataset_group.attrs.create('type', [b"US"], dtype='|S2')
    US_dataset_group.attrs.create('subtype', [b"CPW"], dtype='|S3')
    US_dataset_group.attrs.create('signal_format', [b"RF"], dtype='|S2')
    US_dataset_group.attrs.create('name', [b"CIRS 040GSE Wires"], dtype='|S18')
    US_dataset_group.attrs.create('version', [b"v1.96"], dtype='|S6')
    US_dataset_group.attrs.create('creation_date', [b"2016/03/9 17:25:22.38"], dtype='|S22')

    # angles
    US_dataset_group.create_dataset( 'angles', data=angles)

    # PRF
    US_dataset_group.create_dataset( 'PRF', data=PRF)

    # initial_time
    US_dataset_group.create_dataset( 'initial_time', data=initial_time)

    # modulation frequency
    US_dataset_group.create_dataset( 'modulation_frequency', data=modulation_frequency)

    # probe geometry
    US_dataset_group.create_dataset( 'probe_geometry', data=probe_geometry)

    # sampling frequency
    US_dataset_group.create_dataset( 'sampling_frequency', data=sampling_frequency)

    # sound speed
    US_dataset_group.create_dataset( 'sound_speed', data=sound_speed)

    # imag data
    data_group = US_dataset_group.require_group('data')
    data_group.create_dataset('imag', data=imag_np)

    # real data
    data_group.create_dataset('real', data=reconstr_data)

    comp_file.close()

In [11]:
# 2
# ADC -> Dec/Dem -> LAVA

# Interpolation -> Interpolation/Modulation

def Dec_Dem_LAVA():

    dataset = 'contrast_speckle'
    threshold = 0.00193345

    f_sampling = 20.832 * (10 ** 6)
    f_carrier = 5.2083 * (10 ** 6)
    compression_ratio = 8

    # Get input data
    rf_original_data = get_original_rf_data("../datasets/archive/archive_to_download/database/experiments/" + dataset + "/rf_dataset.hdf5")

    # High-pass filter data before conversion
    #rf_data_high_pass = filter_band_pass(rf_original_data, f_sampling, f_sampling/20.832, f_sampling/2.5, f_sampling/100)

    # Decimate and demodulate for all shots
    compr_data = np.ndarray(shape=(rf_original_data.shape[0], rf_original_data.shape[1], int(rf_original_data.shape[2]/compression_ratio)), dtype=np.complex64)

    for shot in range(0, rf_original_data.shape[0]):
        compr_data[shot] = demodulate_decimate(rf_original_data[shot], f_sampling, f_carrier, compression_ratio)

    #print('Compressed data shape: ', compr_data.shape)

    # LAVA
    kept_val_LAVA, indexes_LAVA = LAVA(threshold, compr_data.real)
    kept_val_imag_LAVA, indexes_LAVA_imag = LAVA(threshold, compr_data.imag)

    # Inverse LAVA / Interpolate
    range_of_samples = [range(0,compr_data.shape[2])]
    reconstr_data_real = np.zeros((75, 128, compr_data.shape[2]), dtype=np.float32)
    reconstr_data_imag = np.zeros((75, 128, compr_data.shape[2]), dtype=np.float32)
    no_samples = 0
    no_indexes = 0
    no_samples_imag = 0
    no_indexes_imag = 0
    
    for i in range(len(kept_val_LAVA)):
        for k in range(len(kept_val_LAVA[0])):
            
            # Calculate number of samples transmitted
            no_samples = no_samples + len(kept_val_LAVA[i][k])
            no_indexes = no_indexes + len(indexes_LAVA[i][k])
            
            no_samples_imag = no_samples_imag + len(kept_val_imag_LAVA[i][k])
            no_indexes_imag = no_indexes_imag + len(indexes_LAVA_imag[i][k])
            
            reconstr_data_real[i][k] = np.interp(range_of_samples, indexes_LAVA[i][k], kept_val_LAVA[i][k])
            reconstr_data_imag[i][k] = np.interp(range_of_samples, indexes_LAVA_imag[i][k], kept_val_imag_LAVA[i][k])


    # Generate complex number from the above arrays
    import cmath

    complex_numb = np.empty((75, 128, compr_data.shape[2]), dtype=np.complex64)
    complex_numb.real = reconstr_data_real
    complex_numb.imag = reconstr_data_imag

    # Interpolate / Modulate
    inter_data = np.zeros(rf_original_data.shape, dtype=np.complex64)
    for shot in range(complex_numb.shape[0]):
        inter_data[shot] = interpolate_modulate(complex_numb[shot], f_sampling/compression_ratio, f_carrier, compression_ratio)

        
    # Calculate compression ratio
    # Transfer demodulated data and all the indexes as they are after LAVA
    # Number of bits needed for indexes varies according to compression ratio of decimation/demodulation
    no_bits_indexes = int(np.ceil(np.log2(compr_data.shape[2])))
    initial_number_of_bits = (rf_original_data.size)*12
    compressed_number_of_bits = 12*(no_samples) + no_bits_indexes*(no_indexes) + 12*(no_samples_imag) + no_bits_indexes*(no_indexes_imag)
    
    # Compression ratio
    compr_ratio = (compressed_number_of_bits/initial_number_of_bits)*100
    print("Dec_Dem_LAVA is {:.2f}% of the original data.".format(compr_ratio))
    
    
    # Beamform and plot the compressed image

    dataset_path = "../datasets/archive/archive_to_download/database/experiments/" + dataset + "/rf_dataset.hdf5"

    data_loader_obj = DataLoader(dataset_path)

    ### Specify Image settings and create corresponding object ###

    img_res = [400, 600]
    image_x_range = [-0.019, 0.019]
    image_z_range = [0.005, 0.05]

    db_range = 50

    LATERAL_PIXEL_DENSITY_DEFAULT = 5

    img_config = ImageSettings(image_x_range[0],
                               image_x_range[1],
                               image_z_range[0],
                               image_z_range[1],
                               LATERAL_PIXEL_DENSITY_DEFAULT,
                               data_loader_obj.transducer)

    ### Specify preprocessing parameters for RF data ###

    decimation_factor = 1
    interpolation_factor = 10

    ### Specify TX strategy and Apodization parameters ###

    start_time = 0
    correction_time_shift = 0

    alpha_fov_apod = 40

    ### Specify Sampling Frequency ###

    SAMPLING_FREQ = 20.832 * (10 ** 6)

    filters_params = [1 * 10 **6, 8 * 10 **6, 0.5 * 10 **6]

    bf = BFCartesianRealTime(data_loader_obj.f_sampling,
                             data_loader_obj.tx_strategy,
                             data_loader_obj.transducer,
                             decimation_factor,
                             interpolation_factor,
                             img_res,
                             img_config,
                             start_time=start_time,
                             correction_time_shift=correction_time_shift,
                             alpha_fov_apod=alpha_fov_apod,
                             bp_filter_params=filters_params,
                             envelope_detector='hilbert',
                             picmus_dataset=True)


    # Beamforming
    img_data = bf.beamform(inter_data.real, numba_active=True)

    _ = plot_image(np.abs(img_data), 
                   scatters_coords_xz=None,
                   elements_coords_xz=None,
                   framework='plotly',
                   title='Dec_Dem_LAVA: {:.2f}% of the original data'.format(compr_ratio),
                   image_x_range=image_x_range,
                   image_z_range=image_z_range,
                   db_range=db_range,
                   colorscale='Greys',
                   save_fig=True, 
                   show=True,
                   path_to_save='./Dec_Dem_LAVA')
    
    # Prepare dataset to extract metrics of resolution and contrast

    # Read info from file
    f_d = h5py.File("../datasets/archive/archive_to_download/database/experiments/"+dataset+"/"+dataset+"_expe_dataset_rf.hdf5", "r")

    attributes = f_d.attrs

    US = f_d['US']
    attributes = US.attrs

    US_DATASET0000 = US['US_DATASET0000']
    attributes = US_DATASET0000.attrs

    type_s = attributes.__getitem__('type')

    attributes = US_DATASET0000.attrs

    keys_1 = list(f_d.keys());

    US = f_d['US']

    US_DATASET0000 = US['US_DATASET0000']

    data = US_DATASET0000['data']

    imag_n = data['imag']

    real_n = data['real'][()]

    PRF = US_DATASET0000['PRF']

    angles = US_DATASET0000['angles']

    initial_time = US_DATASET0000['initial_time']

    modulation_frequency = US_DATASET0000['modulation_frequency']

    probe_geometry = US_DATASET0000['probe_geometry']

    sampling_frequency = US_DATASET0000['sampling_frequency']

    sound_speed = US_DATASET0000['sound_speed']

    # Create dataset suitable for calculating metrics

    imag_np = np.zeros((75, 128, 3328))

    # Creation of dataset

    out_filename = out_filename = './Dec_Dem_LAVA/Dec_Dem_LAVA_th{}_cr{}.hdf5'.format(threshold,compression_ratio)
    comp_file = h5py.File(out_filename,'w')
    comp_file.close()
    comp_file = h5py.File(out_filename,'a')

    # Complete the structure

    comp_file.attrs.create('version', [b'v.0.0.40'], dtype='|S9')

    US_path = 'US'
    US_group = comp_file.require_group(US_path)

    US_dataset_group = US_group.require_group('US_DATASET0000')

    US_dataset_group.attrs.create('type', [b"US"], dtype='|S2')
    US_dataset_group.attrs.create('subtype', [b"CPW"], dtype='|S3')
    US_dataset_group.attrs.create('signal_format', [b"RF"], dtype='|S2')
    US_dataset_group.attrs.create('name', [b"CIRS 040GSE Wires"], dtype='|S18')
    US_dataset_group.attrs.create('version', [b"v1.96"], dtype='|S6')
    US_dataset_group.attrs.create('creation_date', [b"2016/03/9 17:25:22.38"], dtype='|S22')

    # angles
    US_dataset_group.create_dataset( 'angles', data=angles)

    # PRF
    US_dataset_group.create_dataset( 'PRF', data=PRF)

    # initial_time
    US_dataset_group.create_dataset( 'initial_time', data=initial_time)

    # modulation frequency
    US_dataset_group.create_dataset( 'modulation_frequency', data=modulation_frequency)

    # probe geometry
    US_dataset_group.create_dataset( 'probe_geometry', data=probe_geometry)

    # sampling frequency
    US_dataset_group.create_dataset( 'sampling_frequency', data=sampling_frequency)

    # sound speed
    US_dataset_group.create_dataset( 'sound_speed', data=sound_speed)

    # imag data
    data_group = US_dataset_group.require_group('data')
    data_group.create_dataset('imag', data=imag_np)

    # real data
    data_group.create_dataset('real', data=inter_data.real)

    comp_file.close()


In [12]:
# 5
# ADC -> Requantization -> Decimation/Demodulation

# Interpolation/Modulation -> Inverse Requantization

def Requant_Dec_Dem():

    dataset = 'contrast_speckle'
    threshold = 0.00193345

    f_sampling = 20.832 * (10 ** 6)
    f_carrier = 5.2083 * (10 ** 6)

    compression_ratio = 8

    # Get input data
    rf_original_data = get_original_rf_data("../datasets/archive/archive_to_download/database/experiments/" + dataset + "/rf_dataset.hdf5")

    with open('./BL_Coding/'+dataset+'/int_converted_values.npy', 'rb') as f:
        int_out_values = np.load(f)
    f.close()

    # Requantize
    with open('./Requantization/'+dataset+'_forw_requant_9_data.npy', 'rb') as f:
        requant_data = np.load(f)
    f.close()

    # Convert to floats
    divider = ((2**9)-1)/2
    rf_range_data = requant_data/divider
    rf_data = rf_range_data - 1

    # Decimation and Demodulation
    compr_data = np.ndarray(shape=(rf_data.shape[0], rf_original_data.shape[1], int(rf_original_data.shape[2]/compression_ratio)), dtype=np.complex64)

    for shot in range(0, rf_data.shape[0]):
        compr_data[shot] = demodulate_decimate(rf_data[shot], f_sampling, f_carrier, compression_ratio)

    print('Compressed data shape: ', compr_data.shape)
    print('Done with Dem/Dec.')
    

    # Interpolate and Modulate
    inter_data = np.zeros(rf_data.shape, dtype=np.complex64)
    for shot in range(0, compr_data.shape[0]):
        inter_data[shot] = interpolate_modulate(compr_data[shot], f_sampling/compression_ratio, f_carrier, compression_ratio)

    print('Interpolated data shape: ', inter_data.shape)


    # Inverse Requantization
    inter_data_real = inter_data.real

    # Convert to integers
    inter_integers = convert_to_int(inter_data_real, 9, 2)

    inv_req_real = inverse_requantization(inter_integers,0,1800,0,44,1800,2200,45,444,2200,4095,445,511)

    # Compression ratio
    initial_number_of_bits = (rf_original_data.size)*12
    compressed_number_of_bits = 18*(compr_data.size)    # Need to transfer both real and imaginary parts of the samples
    
    # Compression ratio
    compr_ratio = (compressed_number_of_bits/initial_number_of_bits)*100
    print("Requant_Dec_Dem is {:.2f}% of the original data.".format(compr_ratio))
    
    
    # Beamform and plot the compressed image

    dataset_path = "../datasets/archive/archive_to_download/database/experiments/" + dataset + "/rf_dataset.hdf5"

    data_loader_obj = DataLoader(dataset_path)

    ### Specify Image settings and create corresponding object ###

    img_res = [400, 600]
    image_x_range = [-0.019, 0.019]
    image_z_range = [0.005, 0.05]

    db_range = 50

    LATERAL_PIXEL_DENSITY_DEFAULT = 5

    img_config = ImageSettings(image_x_range[0],
                               image_x_range[1],
                               image_z_range[0],
                               image_z_range[1],
                               LATERAL_PIXEL_DENSITY_DEFAULT,
                               data_loader_obj.transducer)

    ### Specify preprocessing parameters for RF data ###

    decimation_factor = 1
    interpolation_factor = 10

    ### Specify TX strategy and Apodization parameters ###

    start_time = 0
    correction_time_shift = 0

    alpha_fov_apod = 40

    ### Specify Sampling Frequency ###

    SAMPLING_FREQ = 20.832 * (10 ** 6)

    filters_params = [1 * 10 **6, 8 * 10 **6, 0.5 * 10 **6]

    bf = BFCartesianRealTime(data_loader_obj.f_sampling,
                             data_loader_obj.tx_strategy,
                             data_loader_obj.transducer,
                             decimation_factor,
                             interpolation_factor,
                             img_res,
                             img_config,
                             start_time=start_time,
                             correction_time_shift=correction_time_shift,
                             alpha_fov_apod=alpha_fov_apod,
                             bp_filter_params=filters_params,
                             envelope_detector='hilbert',
                             picmus_dataset=True)


    # Beamforming
    img_data = bf.beamform(inv_req_real, numba_active=True)

    _ = plot_image(np.abs(img_data), 
                   scatters_coords_xz=None,
                   elements_coords_xz=None,
                   framework='plotly',
                   title='Requant_Dec_Dem: {:.2f}% of the original data'.format(compr_ratio),
                   image_x_range=image_x_range,
                   image_z_range=image_z_range,
                   db_range=db_range,
                   colorscale='Greys',
                   save_fig=True, 
                   show=True,
                   path_to_save='./Requant_Dec_Dem')
    
    # Prepare dataset to extract metrics of resolution and contrast

    # Read info from file
    f_d = h5py.File("../datasets/archive/archive_to_download/database/experiments/"+dataset+"/"+dataset+"_expe_dataset_rf.hdf5", "r")

    attributes = f_d.attrs

    US = f_d['US']
    attributes = US.attrs

    US_DATASET0000 = US['US_DATASET0000']
    attributes = US_DATASET0000.attrs

    type_s = attributes.__getitem__('type')

    attributes = US_DATASET0000.attrs

    keys_1 = list(f_d.keys());

    US = f_d['US']

    US_DATASET0000 = US['US_DATASET0000']

    data = US_DATASET0000['data']

    imag_n = data['imag']

    real_n = data['real'][()]

    PRF = US_DATASET0000['PRF']

    angles = US_DATASET0000['angles']

    initial_time = US_DATASET0000['initial_time']

    modulation_frequency = US_DATASET0000['modulation_frequency']

    probe_geometry = US_DATASET0000['probe_geometry']

    sampling_frequency = US_DATASET0000['sampling_frequency']

    sound_speed = US_DATASET0000['sound_speed']

    # Create dataset suitable for calculating metrics

    imag_np = np.zeros((75, 128, 3328))

    # Creation of dataset

    out_filename = out_filename = './Requant_Dec_Dem/Requant_Dec_Dem_cr{}.hdf5'.format(compression_ratio)
    comp_file = h5py.File(out_filename,'w')
    comp_file.close()
    comp_file = h5py.File(out_filename,'a')

    # Complete the structure

    comp_file.attrs.create('version', [b'v.0.0.40'], dtype='|S9')

    US_path = 'US'
    US_group = comp_file.require_group(US_path)

    US_dataset_group = US_group.require_group('US_DATASET0000')

    US_dataset_group.attrs.create('type', [b"US"], dtype='|S2')
    US_dataset_group.attrs.create('subtype', [b"CPW"], dtype='|S3')
    US_dataset_group.attrs.create('signal_format', [b"RF"], dtype='|S2')
    US_dataset_group.attrs.create('name', [b"CIRS 040GSE Wires"], dtype='|S18')
    US_dataset_group.attrs.create('version', [b"v1.96"], dtype='|S6')
    US_dataset_group.attrs.create('creation_date', [b"2016/03/9 17:25:22.38"], dtype='|S22')

    # angles
    US_dataset_group.create_dataset( 'angles', data=angles)

    # PRF
    US_dataset_group.create_dataset( 'PRF', data=PRF)

    # initial_time
    US_dataset_group.create_dataset( 'initial_time', data=initial_time)

    # modulation frequency
    US_dataset_group.create_dataset( 'modulation_frequency', data=modulation_frequency)

    # probe geometry
    US_dataset_group.create_dataset( 'probe_geometry', data=probe_geometry)

    # sampling frequency
    US_dataset_group.create_dataset( 'sampling_frequency', data=sampling_frequency)

    # sound speed
    US_dataset_group.create_dataset( 'sound_speed', data=sound_speed)

    # imag data
    data_group = US_dataset_group.require_group('data')
    data_group.create_dataset('imag', data=imag_np)

    # real data
    rf_range_data = inv_req_real/2047.5
    rf_data = rf_range_data - 1
    data_group.create_dataset('real', data=rf_data)

    comp_file.close()



In [18]:
# 6

# ADC -> Decimation/Demodulation -> Requantization

# Inverse Requantization -> Interpolation/Modulation

def Dec_Dem_Requant(c_ratio, req_bits):
    
    dataset = 'contrast_speckle'

    f_sampling = 20.832 * (10 ** 6)
    f_carrier = 5.2083 * (10 ** 6)
    compression_ratio = c_ratio

    # Get input data
    rf_original_data = get_original_rf_data("../datasets/archive/archive_to_download/database/experiments/" + dataset + "/rf_dataset.hdf5")

    # High-pass filter data before conversion
    #rf_data_high_pass = filter_band_pass(rf_original_data, f_sampling, f_sampling/20.832, f_sampling/2.5, f_sampling/100)

    # Decimate and demodulate for all shots
    compr_data = np.ndarray(shape=(rf_original_data.shape[0], rf_original_data.shape[1], int(np.ceil(rf_original_data.shape[2]/compression_ratio))), dtype=np.complex64)

    for shot in range(0, rf_original_data.shape[0]):
        compr_data[shot] = demodulate_decimate(rf_original_data[shot], f_sampling, f_carrier, compression_ratio)

    print('Compressed data shape: ', compr_data.shape)

    # Requantization
    int_real = convert_to_int_2(compr_data.real, 10, 2)
    int_imag = convert_to_int_2(compr_data.imag, 10, 2)

    # Reduce values to 9 bits
    requant_bits = req_bits
    divider = 512/(2**(req_bits-1))
    int_9_real = np.floor(int_real/divider)
    int_9_real = int_9_real.astype(int)

    # Reduce values to 9 bits
    int_9_imag = np.floor(int_imag/divider)
    int_9_imag = int_9_imag.astype(int)

    # Generate complex number from the above arrays
    import cmath

    complex_numb = np.empty((75, 128, compr_data.shape[2]), dtype=np.complex64)
    complex_numb.real = int_9_real
    complex_numb.imag = int_9_imag

    # Interpolate / Modulate
    inter_data = np.zeros(shape=(rf_original_data.shape[0], rf_original_data.shape[1], int(np.ceil(rf_original_data.shape[2]/compression_ratio))*compression_ratio), dtype=np.complex64)
    for shot in range(complex_numb.shape[0]):
        inter_data[shot] = interpolate_modulate(complex_numb[shot], f_sampling/compression_ratio, f_carrier, compression_ratio)


    # Compression ratio
    initial_number_of_bits = (rf_original_data.size)*10
    compressed_number_of_bits = (2*requant_bits)*(compr_data.size) # Need to transmit both real and imaginary parts of the samples

    # Compression ratio
    compr_ratio = (compressed_number_of_bits/initial_number_of_bits)*100
    print("Dec_Dem_Requant is {:.2f}% of the original data.".format(compr_ratio))

    # Beamform and plot the compressed image

    dataset_path = "../datasets/archive/archive_to_download/database/experiments/" + dataset + "/rf_dataset.hdf5"

    data_loader_obj = DataLoader(dataset_path)

    ### Specify Image settings and create corresponding object ###

    img_res = [400, 600]
    image_x_range = [-0.019, 0.019]
    image_z_range = [0.005, 0.05]

    db_range = 50

    LATERAL_PIXEL_DENSITY_DEFAULT = 5

    img_config = ImageSettings(image_x_range[0],
                               image_x_range[1],
                               image_z_range[0],
                               image_z_range[1],
                               LATERAL_PIXEL_DENSITY_DEFAULT,
                               data_loader_obj.transducer)

    ### Specify preprocessing parameters for RF data ###

    decimation_factor = 1
    interpolation_factor = 10

    ### Specify TX strategy and Apodization parameters ###

    start_time = 0
    correction_time_shift = 0

    alpha_fov_apod = 40

    ### Specify Sampling Frequency ###

    SAMPLING_FREQ = 20.832 * (10 ** 6)

    filters_params = [1 * 10 **6, 8 * 10 **6, 0.5 * 10 **6]

    bf = BFCartesianRealTime(data_loader_obj.f_sampling,
                             data_loader_obj.tx_strategy,
                             data_loader_obj.transducer,
                             decimation_factor,
                             interpolation_factor,
                             img_res,
                             img_config,
                             start_time=start_time,
                             correction_time_shift=correction_time_shift,
                             alpha_fov_apod=alpha_fov_apod,
                             bp_filter_params=filters_params,
                             envelope_detector='hilbert',
                             picmus_dataset=True)


    # Beamforming
    img_data = bf.beamform(inter_data.real, numba_active=True)

    _ = plot_image(np.abs(img_data), 
                   scatters_coords_xz=None,
                   elements_coords_xz=None,
                   framework='plotly',
                   title='Dec_Dem_Requant_{}_{}: {:.2f}% of the original data'.format(c_ratio,req_bits,compr_ratio),
                   image_x_range=image_x_range,
                   image_z_range=image_z_range,
                   db_range=db_range,
                   colorscale='Greys',
                   save_fig=True, 
                   show=True,
                   path_to_save='./Dec_Dem_Requant')
    
    # Read info from file
    f_d = h5py.File("../datasets/archive/archive_to_download/database/experiments/"+dataset+"/"+dataset+"_expe_dataset_rf.hdf5", "r")

    attributes = f_d.attrs

    US = f_d['US']
    attributes = US.attrs

    US_DATASET0000 = US['US_DATASET0000']
    attributes = US_DATASET0000.attrs

    type_s = attributes.__getitem__('type')

    attributes = US_DATASET0000.attrs

    keys_1 = list(f_d.keys());

    US = f_d['US']

    US_DATASET0000 = US['US_DATASET0000']

    data = US_DATASET0000['data']

    imag_n = data['imag']

    real_n = data['real'][()]

    PRF = US_DATASET0000['PRF']

    angles = US_DATASET0000['angles']

    initial_time = US_DATASET0000['initial_time']

    modulation_frequency = US_DATASET0000['modulation_frequency']

    probe_geometry = US_DATASET0000['probe_geometry']

    sampling_frequency = US_DATASET0000['sampling_frequency']

    sound_speed = US_DATASET0000['sound_speed']

    # Create dataset suitable for calculating metrics

    imag_np = np.zeros(shape = inter_data.real.shape)

    # Creation of dataset

    out_filename = out_filename = './Dec_Dem_Requant/Dec_Dem_Requant_{}_{}.hdf5'.format(c_ratio,req_bits)
    comp_file = h5py.File(out_filename,'w')
    comp_file.close()
    comp_file = h5py.File(out_filename,'a')

    # Complete the structure

    comp_file.attrs.create('version', [b'v.0.0.40'], dtype='|S9')

    US_path = 'US'
    US_group = comp_file.require_group(US_path)

    US_dataset_group = US_group.require_group('US_DATASET0000')

    US_dataset_group.attrs.create('type', [b"US"], dtype='|S2')
    US_dataset_group.attrs.create('subtype', [b"CPW"], dtype='|S3')
    US_dataset_group.attrs.create('signal_format', [b"RF"], dtype='|S2')
    US_dataset_group.attrs.create('name', [b"CIRS 040GSE Wires"], dtype='|S18')
    US_dataset_group.attrs.create('version', [b"v1.96"], dtype='|S6')
    US_dataset_group.attrs.create('creation_date', [b"2016/03/9 17:25:22.38"], dtype='|S22')

    # angles
    US_dataset_group.create_dataset( 'angles', data=angles)

    # PRF
    US_dataset_group.create_dataset( 'PRF', data=PRF)

    # initial_time
    US_dataset_group.create_dataset( 'initial_time', data=initial_time)

    # modulation frequency
    US_dataset_group.create_dataset( 'modulation_frequency', data=modulation_frequency)

    # probe geometry
    US_dataset_group.create_dataset( 'probe_geometry', data=probe_geometry)

    # sampling frequency
    US_dataset_group.create_dataset( 'sampling_frequency', data=sampling_frequency)

    # sound speed
    US_dataset_group.create_dataset( 'sound_speed', data=sound_speed)

    # imag data
    data_group = US_dataset_group.require_group('data')
    data_group.create_dataset('imag', data=imag_np)

    # real data
    data_group.create_dataset('real', data=inter_data.real)

    comp_file.close()
    
    return int_9_real, int_9_imag

In [14]:
# 9
# ADC -> LAVA -> Requantization

# Inverse Requantization -> Interpolation

def LAVA_Requant():

    dataset = 'contrast_speckle'
    threshold = 2

    f_sampling = 20.832 * (10 ** 6)
    f_carrier = 5.2083 * (10 ** 6)

    requant_bits = 9

    # Read input integers
    with open('./BL_Coding/'+dataset+'/int_converted_values.npy', 'rb') as f:
        int_out_values = np.load(f)
    f.close()

    # LAVA
    kept_val_LAVA, indexes_LAVA = LAVA(threshold, int_out_values)

    # Arrays for inverse LAVA
    range_of_samples = [range(0,int_out_values.shape[2])]
    reconstr_data_real = np.zeros((75, 128, int_out_values.shape[2]), dtype=np.float32)

    print('Starting Requantization.')
    
    no_samples = 0
    no_indexes = 0

    # Requantization
    # REMEMBER FOR THE INDEXES WE NEED ONLY AS MANY BITS AS LOG2(TOTAL_NUMBER_OF_SAMPLES)
    # Convert individual channel acquisitions from list to array
    for i in range(len(kept_val_LAVA)):
        for k in range(len(kept_val_LAVA[0])):

            current_kept_real = np.zeros((len(kept_val_LAVA[i][k])))
            current_kept_real = np.array(kept_val_LAVA[i][k], dtype=object)

            requant_real = requantization(requant_bits, current_kept_real, 0,1800,0,44,1800,2200,45,444,2200,4095,445,511)

            # Calculate number of samples transmitted
            no_samples = no_samples + len(kept_val_LAVA[i][k])
            no_indexes = no_indexes + len(indexes_LAVA[i][k])
            
            # Inverse Requantization
            inv_req_real = inverse_requantization(requant_real,0,1800,0,44,1800,2200,45,444,2200,4095,445,511)

            # Inverse LAVA
            reconstr_data_real[i][k] = np.interp(range_of_samples, indexes_LAVA[i][k], inv_req_real)
            #sys.stdout.write('\r')
            #sys.stdout.write('Done with Acq {} Chan {}.'.format(i, k))

    # Calculate compression ratio
    initial_number_of_bits = (int_out_values.size)*12
    compressed_number_of_bits = requant_bits*(no_samples) + 12*(no_indexes)
    
    # Compression ratio
    compr_ratio = (compressed_number_of_bits/initial_number_of_bits)*100
    print("LAVA_Requant is {:.2f}% of the original data.".format(compr_ratio))
    
    # Beamform and plot the compressed image

    dataset_path = "../datasets/archive/archive_to_download/database/experiments/" + dataset + "/rf_dataset.hdf5"

    data_loader_obj = DataLoader(dataset_path)

    ### Specify Image settings and create corresponding object ###

    img_res = [400, 600]
    image_x_range = [-0.019, 0.019]
    image_z_range = [0.005, 0.05]

    db_range = 50

    LATERAL_PIXEL_DENSITY_DEFAULT = 5

    img_config = ImageSettings(image_x_range[0],
                               image_x_range[1],
                               image_z_range[0],
                               image_z_range[1],
                               LATERAL_PIXEL_DENSITY_DEFAULT,
                               data_loader_obj.transducer)

    ### Specify preprocessing parameters for RF data ###

    decimation_factor = 1
    interpolation_factor = 10

    ### Specify TX strategy and Apodization parameters ###

    start_time = 0
    correction_time_shift = 0

    alpha_fov_apod = 40

    ### Specify Sampling Frequency ###

    SAMPLING_FREQ = 20.832 * (10 ** 6)

    filters_params = [1 * 10 **6, 8 * 10 **6, 0.5 * 10 **6]

    bf = BFCartesianRealTime(data_loader_obj.f_sampling,
                             data_loader_obj.tx_strategy,
                             data_loader_obj.transducer,
                             decimation_factor,
                             interpolation_factor,
                             img_res,
                             img_config,
                             start_time=start_time,
                             correction_time_shift=correction_time_shift,
                             alpha_fov_apod=alpha_fov_apod,
                             bp_filter_params=filters_params,
                             envelope_detector='hilbert',
                             picmus_dataset=True)


    # Beamforming
    img_data = bf.beamform(reconstr_data_real, numba_active=True)

    _ = plot_image(np.abs(img_data), 
                   scatters_coords_xz=None,
                   elements_coords_xz=None,
                   framework='plotly',
                   title='LAVA_Requant: {:.2f}% of the original data'.format(compr_ratio),
                   image_x_range=image_x_range,
                   image_z_range=image_z_range,
                   db_range=db_range,
                   colorscale='Greys',
                   save_fig=True, 
                   show=True,
                   path_to_save='./LAVA_Requant')
    
    # Read info from file
    f_d = h5py.File("../datasets/archive/archive_to_download/database/experiments/"+dataset+"/"+dataset+"_expe_dataset_rf.hdf5", "r")

    attributes = f_d.attrs

    US = f_d['US']
    attributes = US.attrs

    US_DATASET0000 = US['US_DATASET0000']
    attributes = US_DATASET0000.attrs

    type_s = attributes.__getitem__('type')

    attributes = US_DATASET0000.attrs

    keys_1 = list(f_d.keys());

    US = f_d['US']

    US_DATASET0000 = US['US_DATASET0000']

    data = US_DATASET0000['data']

    imag_n = data['imag']

    real_n = data['real'][()]

    PRF = US_DATASET0000['PRF']

    angles = US_DATASET0000['angles']

    initial_time = US_DATASET0000['initial_time']

    modulation_frequency = US_DATASET0000['modulation_frequency']

    probe_geometry = US_DATASET0000['probe_geometry']

    sampling_frequency = US_DATASET0000['sampling_frequency']

    sound_speed = US_DATASET0000['sound_speed']

    # Create dataset suitable for calculating metrics

    imag_np = np.zeros((75, 128, 3328))

    # Creation of dataset

    out_filename = out_filename = './LAVA_Requant/LAVA_Requant_th{}.hdf5'.format(threshold)
    comp_file = h5py.File(out_filename,'w')
    comp_file.close()
    comp_file = h5py.File(out_filename,'a')

    # Complete the structure

    comp_file.attrs.create('version', [b'v.0.0.40'], dtype='|S9')

    US_path = 'US'
    US_group = comp_file.require_group(US_path)

    US_dataset_group = US_group.require_group('US_DATASET0000')

    US_dataset_group.attrs.create('type', [b"US"], dtype='|S2')
    US_dataset_group.attrs.create('subtype', [b"CPW"], dtype='|S3')
    US_dataset_group.attrs.create('signal_format', [b"RF"], dtype='|S2')
    US_dataset_group.attrs.create('name', [b"CIRS 040GSE Wires"], dtype='|S18')
    US_dataset_group.attrs.create('version', [b"v1.96"], dtype='|S6')
    US_dataset_group.attrs.create('creation_date', [b"2016/03/9 17:25:22.38"], dtype='|S22')

    # angles
    US_dataset_group.create_dataset( 'angles', data=angles)

    # PRF
    US_dataset_group.create_dataset( 'PRF', data=PRF)

    # initial_time
    US_dataset_group.create_dataset( 'initial_time', data=initial_time)

    # modulation frequency
    US_dataset_group.create_dataset( 'modulation_frequency', data=modulation_frequency)

    # probe geometry
    US_dataset_group.create_dataset( 'probe_geometry', data=probe_geometry)

    # sampling frequency
    US_dataset_group.create_dataset( 'sampling_frequency', data=sampling_frequency)

    # sound speed
    US_dataset_group.create_dataset( 'sound_speed', data=sound_speed)

    # imag data
    data_group = US_dataset_group.require_group('data')
    data_group.create_dataset('imag', data=imag_np)

    # real data
    rf_range_data = reconstr_data_real/2047.5
    rf_data = rf_range_data - 1
    data_group.create_dataset('real', data=rf_data)

    comp_file.close()

In [15]:
# 10
# ADC -> Requantization -> LAVA

# Interpolation -> Inverse Requantization

def Requant_LAVA():

    dataset = 'contrast_speckle'
    threshold = 14

    f_sampling = 20.832 * (10 ** 6)
    f_carrier = 5.2083 * (10 ** 6)

    compression_ratio = 8

    # Get input data
    rf_original_data = get_original_rf_data("../datasets/archive/archive_to_download/database/experiments/" + dataset + "/rf_dataset.hdf5")

    with open('./BL_Coding/'+dataset+'/int_converted_values.npy', 'rb') as f:
        int_out_values = np.load(f)
    f.close()

    # Requantize
    with open('./Requantization/'+dataset+'_forw_requant_9_data.npy', 'rb') as f:
        requant_data = np.load(f)
    f.close()

    # LAVA
    kept_val_LAVA, indexes_LAVA = LAVA(threshold, requant_data)

    # Inverse LAVA / Interpolate
    range_of_samples = [range(0,requant_data.shape[2])]
    reconstr_data_real = np.zeros((75, 128, requant_data.shape[2]), dtype=np.float32)
    no_samples = 0
    no_indexes = 0

    for i in range(len(kept_val_LAVA)):
        for k in range(len(kept_val_LAVA[0])):
            no_samples = no_samples + len(kept_val_LAVA[i][k])
            no_indexes = no_indexes + len(indexes_LAVA[i][k])
            reconstr_data_real[i][k] = np.interp(range_of_samples, indexes_LAVA[i][k], kept_val_LAVA[i][k])           


    # Inverse Requantization
    inv_req_real = inverse_requantization(reconstr_data_real,0,1800,0,44,1800,2200,45,444,2200,4095,445,511)

    # Calculate compression ratio
    initial_number_of_bits = (rf_original_data.size)*12
    compressed_number_of_bits = 9*(no_samples) + 12*(no_indexes)
    
    # Compression ratio
    compr_ratio = (compressed_number_of_bits/initial_number_of_bits)*100
    print("Requant_LAVA is {:.2f}% of the original data.".format(compr_ratio))

    # Beamform and plot the compressed image

    dataset_path = "../datasets/archive/archive_to_download/database/experiments/" + dataset + "/rf_dataset.hdf5"

    data_loader_obj = DataLoader(dataset_path)

    ### Specify Image settings and create corresponding object ###

    img_res = [400, 600]
    image_x_range = [-0.019, 0.019]
    image_z_range = [0.005, 0.05]

    db_range = 50

    LATERAL_PIXEL_DENSITY_DEFAULT = 5

    img_config = ImageSettings(image_x_range[0],
                               image_x_range[1],
                               image_z_range[0],
                               image_z_range[1],
                               LATERAL_PIXEL_DENSITY_DEFAULT,
                               data_loader_obj.transducer)

    ### Specify preprocessing parameters for RF data ###

    decimation_factor = 1
    interpolation_factor = 10

    ### Specify TX strategy and Apodization parameters ###

    start_time = 0
    correction_time_shift = 0

    alpha_fov_apod = 40

    ### Specify Sampling Frequency ###

    SAMPLING_FREQ = 20.832 * (10 ** 6)

    filters_params = [1 * 10 **6, 8 * 10 **6, 0.5 * 10 **6]

    bf = BFCartesianRealTime(data_loader_obj.f_sampling,
                             data_loader_obj.tx_strategy,
                             data_loader_obj.transducer,
                             decimation_factor,
                             interpolation_factor,
                             img_res,
                             img_config,
                             start_time=start_time,
                             correction_time_shift=correction_time_shift,
                             alpha_fov_apod=alpha_fov_apod,
                             bp_filter_params=filters_params,
                             envelope_detector='hilbert',
                             picmus_dataset=True)


    # Beamforming
    img_data = bf.beamform(inv_req_real, numba_active=True)

    _ = plot_image(np.abs(img_data), 
                   scatters_coords_xz=None,
                   elements_coords_xz=None,
                   framework='plotly',
                   title='Requant_LAVA: {:.2f}% of the original data'.format(compr_ratio),
                   image_x_range=image_x_range,
                   image_z_range=image_z_range,
                   db_range=db_range,
                   colorscale='Greys',
                   save_fig=True, 
                   show=True,
                   path_to_save='./Requant_LAVA')
    
    # Read info from file
    f_d = h5py.File("../datasets/archive/archive_to_download/database/experiments/"+dataset+"/"+dataset+"_expe_dataset_rf.hdf5", "r")

    attributes = f_d.attrs

    US = f_d['US']
    attributes = US.attrs

    US_DATASET0000 = US['US_DATASET0000']
    attributes = US_DATASET0000.attrs

    type_s = attributes.__getitem__('type')

    attributes = US_DATASET0000.attrs

    keys_1 = list(f_d.keys());

    US = f_d['US']

    US_DATASET0000 = US['US_DATASET0000']

    data = US_DATASET0000['data']

    imag_n = data['imag']

    real_n = data['real'][()]

    PRF = US_DATASET0000['PRF']

    angles = US_DATASET0000['angles']

    initial_time = US_DATASET0000['initial_time']

    modulation_frequency = US_DATASET0000['modulation_frequency']

    probe_geometry = US_DATASET0000['probe_geometry']

    sampling_frequency = US_DATASET0000['sampling_frequency']

    sound_speed = US_DATASET0000['sound_speed']

    # Create dataset suitable for calculating metrics

    imag_np = np.zeros((75, 128, 3328))

    # Creation of dataset

    out_filename = out_filename = './Requant_LAVA/Requant_LAVA_th{}.hdf5'.format(threshold)
    comp_file = h5py.File(out_filename,'w')
    comp_file.close()
    comp_file = h5py.File(out_filename,'a')

    # Complete the structure

    comp_file.attrs.create('version', [b'v.0.0.40'], dtype='|S9')

    US_path = 'US'
    US_group = comp_file.require_group(US_path)

    US_dataset_group = US_group.require_group('US_DATASET0000')

    US_dataset_group.attrs.create('type', [b"US"], dtype='|S2')
    US_dataset_group.attrs.create('subtype', [b"CPW"], dtype='|S3')
    US_dataset_group.attrs.create('signal_format', [b"RF"], dtype='|S2')
    US_dataset_group.attrs.create('name', [b"CIRS 040GSE Wires"], dtype='|S18')
    US_dataset_group.attrs.create('version', [b"v1.96"], dtype='|S6')
    US_dataset_group.attrs.create('creation_date', [b"2016/03/9 17:25:22.38"], dtype='|S22')

    # angles
    US_dataset_group.create_dataset( 'angles', data=angles)

    # PRF
    US_dataset_group.create_dataset( 'PRF', data=PRF)

    # initial_time
    US_dataset_group.create_dataset( 'initial_time', data=initial_time)

    # modulation frequency
    US_dataset_group.create_dataset( 'modulation_frequency', data=modulation_frequency)

    # probe geometry
    US_dataset_group.create_dataset( 'probe_geometry', data=probe_geometry)

    # sampling frequency
    US_dataset_group.create_dataset( 'sampling_frequency', data=sampling_frequency)

    # sound speed
    US_dataset_group.create_dataset( 'sound_speed', data=sound_speed)

    # imag data
    data_group = US_dataset_group.require_group('data')
    data_group.create_dataset('imag', data=imag_np)

    # real data
    rf_range_data = inv_req_real/2047.5
    rf_data = rf_range_data - 1
    data_group.create_dataset('real', data=rf_data)

    comp_file.close()

In [16]:
# Flow to run all the algorithms

# See later cells for Decimation-Demodulation -> Re-quantization

# If a progress feedback is required, uncomment the "sys.stdout.write" commands
# in the respective functions

LAVA_Dec_Dem()
Dec_Dem_LAVA()
Requant_Dec_Dem()
LAVA_Requant()
Requant_LAVA()

RF data shape:  (75, 128, 3328)
Done with LAVA compression.
LAVA_Dec_Dem is 50.86% of the original data.
The highest resolution for the system is:  (633, 205)
Delays precalculation...
TX strategy: plane waves
Number of plane waves:  75
Maximum angle:  16.0 °
Apodization precalculation...
Beamforming...
 
Time of execution: 42.14832282066345 seconds
BF Final dB range (-50.0,0.0)
RF data shape:  (75, 128, 3328)
Done with LAVA compression.
Done with LAVA compression.
Dec_Dem_LAVA is 12.12% of the original data.
The highest resolution for the system is:  (633, 205)
Delays precalculation...
TX strategy: plane waves
Number of plane waves:  75
Maximum angle:  16.0 °
Apodization precalculation...
Beamforming...
 
Time of execution: 40.27096509933472 seconds
BF Final dB range (-50.0,0.0)
RF data shape:  (75, 128, 3328)
Compressed data shape:  (75, 128, 416)
Done with Dem/Dec.
Interpolated data shape:  (75, 128, 3328)
Converted data shape: (75, 128, 3328)
Requant_Dec_Dem is 18.75% of the origina

In [35]:
# Decimation-Demodulation -> Re-quantization -> LAVA
# def Dec_Dem_Requant(compr_ratio, req_bits)

int_r_1_16, int_i_1_16 = Dec_Dem_Requant(16,1)
int_r_2_16, int_i_2_16 = Dec_Dem_Requant(16,2)
int_r_3_16, int_i_3_16 = Dec_Dem_Requant(16,3)
int_r_4_16, int_i_4_16 = Dec_Dem_Requant(16,4)
int_r_5_16, int_i_5_16 = Dec_Dem_Requant(16,5)
int_r_6_16, int_i_6_16 = Dec_Dem_Requant(16,6)
int_r_7_16, int_i_7_16 = Dec_Dem_Requant(16,7)
int_r_8_16, int_i_8_16 = Dec_Dem_Requant(16,8)
int_r_9_16, int_i_9_16 = Dec_Dem_Requant(16,9)
int_r_1_32, int_i_1_32 = Dec_Dem_Requant(32,1)
int_r_2_32, int_i_2_32 = Dec_Dem_Requant(32,2)
int_r_3_32, int_i_3_32 = Dec_Dem_Requant(32,3)
int_r_4_32, int_i_4_32 = Dec_Dem_Requant(32,4)
int_r_5_32, int_i_5_32 = Dec_Dem_Requant(32,5)
int_r_6_32, int_i_6_32 = Dec_Dem_Requant(32,6)
int_r_7_32, int_i_7_32 = Dec_Dem_Requant(32,7)
int_r_8_32, int_i_8_32 = Dec_Dem_Requant(32,8)
int_r_9_32, int_i_9_32 = Dec_Dem_Requant(32,9)

RF data shape:  (75, 128, 3328)
Compressed data shape:  (75, 128, 208)
Converted data shape: (75, 128, 208)
Converted data shape: (75, 128, 208)
Dec_Dem_Requant is 6.25% of the original data.
The highest resolution for the system is:  (633, 205)
Delays precalculation...
TX strategy: plane waves
Number of plane waves:  75
Maximum angle:  16.0 °
Apodization precalculation...
Beamforming...
 
Time of execution: 43.651445627212524 seconds
BF Final dB range (-50.0,0.0)


'\nint_r_1_16, int_i_1_16 = Dec_Dem_Requant(16,1)\nint_r_2_16, int_i_2_16 = Dec_Dem_Requant(16,2)\nint_r_3_16, int_i_3_16 = Dec_Dem_Requant(16,3)\nint_r_4_16, int_i_4_16 = Dec_Dem_Requant(16,4)\nint_r_5_16, int_i_5_16 = Dec_Dem_Requant(16,5)\nint_r_6_16, int_i_6_16 = Dec_Dem_Requant(16,6)\nint_r_7_16, int_i_7_16 = Dec_Dem_Requant(16,7)\nint_r_8_16, int_i_8_16 = Dec_Dem_Requant(16,8)\nint_r_9_16, int_i_9_16 = Dec_Dem_Requant(16,9)\nint_r_1_32, int_i_1_32 = Dec_Dem_Requant(32,1)\nint_r_2_32, int_i_2_32 = Dec_Dem_Requant(32,2)\nint_r_3_32, int_i_3_32 = Dec_Dem_Requant(32,3)\nint_r_4_32, int_i_4_32 = Dec_Dem_Requant(32,4)\nint_r_5_32, int_i_5_32 = Dec_Dem_Requant(32,5)\nint_r_6_32, int_i_6_32 = Dec_Dem_Requant(32,6)\nint_r_7_32, int_i_7_32 = Dec_Dem_Requant(32,7)\nint_r_8_32, int_i_8_32 = Dec_Dem_Requant(32,8)\nint_r_9_32, int_i_9_32 = Dec_Dem_Requant(32,9)\n\n'